In [1]:
import numpy as np 
import os
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

In [2]:
cone_data = pd.read_csv('cone.csv', delimiter=r"\s+")
cone_data = cone_data.drop(columns=['arg'])
cylinder_data = pd.read_csv('cylinder.csv', delimiter=r"\s+")
cylinder_data = cylinder_data.drop(columns=['arg'])

cone_target = np.ones((91, 1), dtype=int)
cylinder_target = cone_target + 1
target = np.concatenate((cone_target, cylinder_target))
union_data = pd.concat([cone_data, cylinder_data])
union_data.insert(loc=union_data.shape[1], column="class", value=target)

union_data.head()

,HH.Re,HH.Im,HV.Re,HV.Im,VH.Re,VH.Im,VV.Re,VV.Im,class
0,0.956940,0.185970,0.033606,-0.024850,-0.03361,0.024850,-0.99575,-0.15728,1
1,0.877020,0.261629,0.040545,-0.015380,-0.04055,0.015376,-0.91998,-0.24330,1
2,0.783459,0.324072,0.044947,-0.004260,-0.04495,0.004258,-0.82781,-0.31676,1
3,0.681292,0.372362,0.043603,0.005807,-0.04360,-0.005810,-0.72802,-0.37866,1
4,0.571329,0.403099,0.042011,0.018000,-0.04201,-0.018000,-0.61354,-0.41926,1


In [69]:
X = union_data.drop(columns=['class'])
y = union_data['class']

In [95]:
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import KernelPCA

In [84]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

In [85]:
# Обучим SVM на непреобразованных данных 

from sklearn.svm import SVC

svm = SVC(kernel='linear')
svm.fit(X_train, y_train)
predictions = svm.predict(X_test)

print('Accuracy: {:.2f}'.format(accuracy_score(y_true=y_test, y_pred=predictions)))


Accuracy: 0.71


In [93]:
svm = SVC()

params = {'kernel': ['poly', 'rbf', 'sigmoid'], 'C': np.linspace(0.1, 100, 1000)}
gc = GridSearchCV(estimator=svm, param_grid=params, scoring='accuracy', cv=5)

gc.fit(X_train, y_train)

print('The best accuracy : {:.3f}'.format(gc.best_score_))
print('The best params: C = {:.1f}, kernel = {:}'.format(gc.best_params_['C'], gc.best_params_['kernel']))

The best accuracy : 0.787
The best params: C = 38.7, kernel = rbf


In [94]:
svm = SVC(kernel='rbf', C=38)

svm.fit(X_train, y_train)

predictions = svm.predict(X_test)
print('Accuracy: {:.2f}'.format(accuracy_score(y_true=y_test, y_pred=predictions)))

Accuracy: 0.80


In [141]:
# Преобразуем данные 

kpca = KernelPCA(kernel='rbf', gamma=1000)
X_train_kpca = kpca.fit_transform(X_train)
X_test_kpca = kpca.transform(X_test)

In [142]:
svm = SVC(kernel='linear')
svm.fit(X_train_kpca, y_train)
predictions = svm.predict(X_test_kpca)

print('Accuracy: {:.2f}'.format(accuracy_score(y_true=y_test, y_pred=predictions)))



Accuracy: 0.73


In [148]:
svm = SVC()

params = {'kernel': ['poly', 'rbf', 'sigmoid'], 'C': np.linspace(0.1, 100, 1000)}
gc = GridSearchCV(estimator=svm, param_grid=params, scoring='accuracy', cv=5)

gc.fit(X_train, y_train)

print('The best accuracy : {:.3f}'.format(gc.best_score_))
print('The best params: C = {:.1f}, kernel = {:}'.format(gc.best_params_['C'], gc.best_params_['kernel']))

The best accuracy : 0.858
The best params: C = 67.3, kernel = rbf


In [162]:
svm = SVC(kernel='rbf', C=67.3)

svm.fit(X_train_kpca, y_train)

predictions = svm.predict(X_test_kpca)
print('Accuracy: {:.2f}'.format(accuracy_score(y_true=y_test, y_pred=predictions)))

Accuracy: 0.73
